In this notebook, you should implement a first version of a working machine learning model to predict the age of an Abalone.

A few guidelines:
- The model does not have to be complex. A simple linear regression model is enough.
- You should use MLflow to track your experiments. You can use the MLflow UI to compare your experiments.
- Do not push any MLflow data to the repository. Only the code to run the experiments is interesting and should be pushed.

# import 

In [1]:
import kagglehub
import os
import pandas as pd

/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
# Download latest version
path = kagglehub.dataset_download("rodolfomendes/abalone-dataset")
csv_file_path = os.path.join(path, "abalone.csv")
df = pd.read_csv(csv_file_path)
df.head(10)

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
5,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8
6,F,0.530,0.415,0.150,0.7775,0.2370,0.1415,0.330,20
7,F,0.545,0.425,0.125,0.7680,0.2940,0.1495,0.260,16
8,M,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.165,9
9,F,0.550,0.440,0.150,0.8945,0.3145,0.1510,0.320,19


In [7]:
df.columns

Index(['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight',
       'Viscera weight', 'Shell weight', 'Rings'],
      dtype='object')

Modelling

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

In [11]:
# Encode binary feature
label_encoder = LabelEncoder()
df['Sex_encoded'] = label_encoder.fit_transform(df['Sex'])

# Independent variables (X) and target variable (y)
X = df.drop(['Rings', 'Sex'], axis=1)  
y = df['Rings']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train 
rf_model.fit(X_train, y_train)

# predictions
y_pred = rf_model.predict(X_test)

# RMSE as the metric
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 2.2682538127051997


# Usw MLflow to track versions

In [14]:
import mlflow
from mlflow import MlflowClient

In [16]:
client = MlflowClient()
# Set the experiment name
mlflow.set_experiment("Try different parameters")

In [23]:
# Start a run

n_est_list = [50, 100, 150, 200, 250, 300]

for i in range(len(n_est_list)):
    with mlflow.start_run(run_name=f"parameter {i}") as run:
        run_id = run.info.run_id

        mlflow.set_tag("model_type", "RandomForestRegression")
        mlflow.set_tag("user", "group4")

        # Train model
        model = rf_model = RandomForestRegressor(n_estimators=n_est_list[i], random_state=42)
        model.fit(X_train, y_train)

        mlflow.log_param("n_estimators", model.n_estimators)

        # Evaluate model on training set
        y_pred_train = model.predict(X_train)
        rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
        mlflow.log_metric("rmse_train", rmse_train)

        # Evaluate model on test set
        y_pred_test = model.predict(X_test)
        rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
        mlflow.log_metric("rmse_test", rmse_test)

        # Log your model
        mlflow.sklearn.log_model(model, "RandomForest Regression")

        # Register your model in MLflow model registry
        mlflow.register_model(
            "runs:/"+run_id+"/random_forest_regression", f"est_num {n_est_list[i]}"
        )


/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:16: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/anaconda3/envs/foo-env/lib/python3.10/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'est_num 50'.
2024/10/24 12:13:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 se

In [24]:
!mlflow ui

[2024-10-24 12:13:24 +0200] [38869] [INFO] Starting gunicorn 20.1.0
[2024-10-24 12:13:24 +0200] [38869] [INFO] Listening at: http://127.0.0.1:5000 (38869)
[2024-10-24 12:13:24 +0200] [38869] [INFO] Using worker: sync
[2024-10-24 12:13:24 +0200] [38870] [INFO] Booting worker with pid: 38870
[2024-10-24 12:13:24 +0200] [38871] [INFO] Booting worker with pid: 38871
[2024-10-24 12:13:24 +0200] [38872] [INFO] Booting worker with pid: 38872
[2024-10-24 12:13:24 +0200] [38873] [INFO] Booting worker with pid: 38873
^C
[2024-10-24 12:15:54 +0200] [38869] [INFO] Handling signal: int
[2024-10-24 12:15:54 +0200] [38871] [INFO] Worker exiting (pid: 38871)
[2024-10-24 12:15:54 +0200] [38870] [INFO] Worker exiting (pid: 38870)
[2024-10-24 12:15:54 +0200] [38873] [INFO] Worker exiting (pid: 38873)
[2024-10-24 12:15:54 +0200] [38872] [INFO] Worker exiting (pid: 38872)
